# Imports

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt

from sklearn.impute import KNNImputer
from verstack import NaNImputer
from pyod.models.iforest import IForest
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import RepeatedKFold, GridSearchCV


from lightgbm import LGBMClassifier
from sklearn.ensemble import StackingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import RidgeClassifier, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

In [ ]:
import warnings
warnings.filterwarnings('ignore')  # do not show warnings

# Helpers

## Model related

In [5]:
def test_vanilla(X, y):
    ETC = ExtraTreesClassifier(random_state=42, n_jobs=-1)
    RFC = RandomForestClassifier(random_state=42, n_jobs=-1)
    GBC = GradientBoostingClassifier(random_state=42)
    #LGBMC = LGBMClassifier(random_state=42, n_jobs=-1)
    #XGBC = XGBClassifier(seed=42)
    CBC = CatBoostClassifier(random_state=42, verbose=0)
    KNN = KNeighborsClassifier(n_jobs=-1)
    SGDC = SGDClassifier(random_state=42, n_jobs=-1)
    MLP = MLPClassifier(random_state=42)

    names = ['ETC', 'RFC', 'GBC', 'CBC', 'KNN', 'SGDC', "MLP"]
    models = [ETC, RFC, GBC, CBC, KNN, SGDC, MLP]

    for name, model in zip(names, models):
        metric = 'Accuracy'
        cv = RepeatedKFold(n_splits=5, 
                          n_repeats=1, 
                          random_state=42)
        grid = GridSearchCV(estimator=model, 
                            param_grid={}, 
                            scoring=metric.lower(),
                            cv=cv, 
                            verbose=0,
                            n_jobs=-1)
        grid.fit(X, y)
        print(f"{metric} for {name}: {grid.best_score_}")

In [6]:
def get_feature_importances(X, 
                            y, 
                            max_features: int) -> dict:
    ETC = ExtraTreesClassifier(random_state=42, n_jobs=-1)
    RFC = RandomForestClassifier(random_state=42, n_jobs=-1)
    GBC = GradientBoostingClassifier(random_state=42)
    #LGBMC = LGBMClassifier(random_state=42, n_jobs=-1)
    #XGBC = XGBClassifier(seed=42)
    CBC = CatBoostClassifier(random_state=42, verbose=0)
    SGDC = SGDClassifier(random_state=42, n_jobs=-1)
    MLP = MLPClassifier(random_state=42)

    names = ['ETC', 'RFC', 'GBC', 'CBC', 'SGDC', "MLP"]
    models = [ETC, RFC, GBC, CBC, SGDC, MLP]

    feature_importances_0 = np.zeros((7, max_features))
    feature_importances_1 = np.zeros((7, max_features//2))
    feature_importances_2 = np.zeros((7, max_features//4))
    
    for i, name, model in zip(range(len(models)), names, models):
        print(f'Fitting {name}...', end=' ')
        model.fit(X, y)
        print('Done')
        selector_0 = SelectFromModel(model, 
                                     threshold=-np.inf,
                                     prefit=True,
                                     max_features=max_features)
        feature_importances_0[i] = selector_0.get_support(indices=True)

        selector_1 = SelectFromModel(model, 
                                     threshold=-np.inf,
                                     prefit=True,
                                     max_features=max_features//2)
        feature_importances_1[i] = selector_1.get_support(indices=True)

        selector_2 = SelectFromModel(model, 
                                     threshold=-np.inf,
                                     prefit=True,
                                     max_features=max_features//4)
        feature_importances_2[i] = selector_2.get_support(indices=True)

    tfi_0 = feature_importances_0.flatten()
    tfi_1 = feature_importances_1.flatten()
    tfi_2 = feature_importances_2.flatten()

    u_0 = np.unique(tfi_0)
    u_1 = np.unique(tfi_1)
    u_2 = np.unique(tfi_2)

    fi_dict = {f'feature_importances_{max_features}': u_0.astype(int),
               f'feature_importances_{max_features//2}': u_1.astype(int),
               f'feature_importances_{max_features//4}': u_2.astype(int)}

    return fi_dict

## Dataset class

In [7]:
class DataSet:
    def __init__(self,
                 path: str = 'ML_Project/dataset',
                 years: list = [2014, 2015, 2016, 2017, 2018],
                 nan_handler: list = ['threshold'],
                 zero_handler: list = ['threshold', 'replace'],
                 imputation_handler: str = 'XGB',
                 outlier_handler: list = ['quantile', 'IForest'],
                 config_dict: dict = {}
                 ):
        self.path = path
        self.years = years
        self.nan_handler = nan_handler
        self.zero_handler = zero_handler
        self.imputation_handler = imputation_handler
        self.outlier_handler = outlier_handler
        self.config_dict = config_dict

        self.df = None
        self.year = None
        self.init_shape = None
        self.class_col = None
        self.sector_col = None
        self.sector_ids = None
        self.sector_mapper = None

        self.dfs = []
        self.intersec_cols = None

    def create_dataset(self):
        # loop over all years
        for year in self.years:
            self.__print_sep(60, '#', '\n')

            # generate single dataframe
            self.prepare_single_dataframe(year)

            # append dataframe
            self.dfs.append(self.df)

        # print seperators
        self.__print_sep(60, '#')
        self.__print_sep(60, '#', '\n')

        # get intersecting columns
        self.get_intersecting_columns()

        # concatenate dataframes to one
        df = self.concat_intersecting_dfs()

        # factorize 'Sector' column
        df = self.factorize_col(df)

        return df

    def prepare_single_dataframe(self, year: int):
        # load data
        self.load_df(year)

        # store and drop 'Class' column
        self.store_drop_class_col()

        # handle NaNs
        self.handle_nans()

        # handle zeros
        self.handle_zeros()

        # factorize 'Sector' column and store its mapper
        self.store_factorize_sector_col()

        # impute NaNs
        self.impute_nans()

        # handle outliers
        self.handle_outliers()

        # drop VAR column
        self.drop_var_col()

        # scale data
        self.scale_data()

        # convert to float 32 and join
        self.convert_join()

        # map 'Sector' back to string representation
        self.map_sector_inv()

    def load_df(self, year: int):
        self.__print_header(f'LOADING {year}')

        self.year = year
        print(f'Loading {year}_Financial_Data.csv into a DataFrame', end=' - ')
        self.df = pd.read_csv(os.path.join(self.path, f'{year}_Financial_Data.csv'),
                              index_col=0)
        print('COMPLETE')
        self.init_shape = self.df.shape

        # drop rows with no info
        print('Dropping rows with NaNs only', end=' - ')
        rows = self.df.shape[0]
        self.df.dropna(how='all', inplace=True)
        print(f'{self.df.shape[0] - rows} rows dropped - COMPLETE')

        self.__print_summary(f'Initial DataFrame shape: {self.df.shape}')

    def store_drop_class_col(self):
        self.class_col = self.df['Class'].astype('int8')
        self.df.drop(columns=['Class'], inplace=True)

    def store_factorize_sector_col(self):
        self.sector_col = self.df['Sector']
        self.sector_ids, self.sector_mapper = pd.factorize(self.df['Sector'])
        self.df['Sector'] = self.sector_ids

    def factorize_col(self, df: pd.DataFrame) -> pd.DataFrame:
        sector_ids, self.sector_mapper = pd.factorize(df['Sector'])
        df['Sector'] = sector_ids

        return df

    def handle_nans(self, round: int = 1):
        if round == 1 and self.nan_handler[0]:
            self.__print_header(f'HANDLING NANS')

        if 'threshold' in self.nan_handler:
            nans_thres = self.config_dict[self.year]['nans_thres']
            print(f'Dropping columns with more than {int(nans_thres * 100)}% NaNs', end=' - ')
            cols = self.df.shape[1]
            self.df = self.df.loc[:, self.df.isnull().mean() < nans_thres]  # drop nans
            print(f'{cols - self.df.shape[1]} columns dropped - COMPLETE')
            self.__print_sep(60, '~', '\n')

    def handle_zeros(self):
        self.__print_header(f'HANDLING ZEROS')

        if 'threshold' in self.zero_handler:
            zeros_thres = self.config_dict[self.year]['zeros_thres']
            print(f'Dropping columns with more than {int(zeros_thres * 100)}% zeros', end=' - ')
            cols = self.df.shape[1]
            self.df = self.df.loc[:, (self.df == 0).mean() < zeros_thres]  # drop zeros
            print(f'{cols - self.df.shape[1]} columns dropped - COMPLETE')

        if 'replace' in self.zero_handler:
            print(f'Replacing zeros with NaNs', end=' - ')
            z_count = (self.df == 0).sum().sum()
            self.df = self.df.replace({0: np.nan})
            z_count_after = (self.df == 0).sum().sum()
            print(f'{z_count - z_count_after} zeros replaced - COMPLETE')

        if 'replace' in self.zero_handler and 'drop_again' in self.zero_handler:
            self.handle_nans(round=2)

        da = self.init_shape[1] - self.df.shape[1]
        sc = self.df.shape
        self.__print_summary(f'Total amount of columns dropped: {da} - Current shape: {sc}')

    def impute_nans(self):
        self.__print_header(f'IMPUTE NANS')

        if self.imputation_handler == 'KNN':
            print(f'Imputing {self.df.isnull().sum().sum()} NaNs with KNN', end=' - ')
            imputer = KNNImputer(n_neighbors=20, weights='distance', metric='nan_euclidean', copy=True)
            np_imp = imputer.fit_transform(self.df)
            self.df = pd.DataFrame(np_imp, columns=self.df.columns, index=self.df.index)
            print(f'{self.df.isnull().sum().sum()} NaNs left - COMPLETE')

        elif self.imputation_handler == 'XGB':
            print(f'Imputing {self.df.isnull().sum().sum()} NaNs with XGB', end=' - ')
            imputer = NaNImputer(verbose=False, multiprocessing_load=1)
            self.df = imputer.impute(self.df)
            print(f'{self.df.isnull().sum().sum()} NaNs left - COMPLETE')
            cols = self.df.shape[1]
            self.df.dropna(axis=1, inplace=True)  # remove columns that still have NaNs inside
            print(f'Removing {cols - self.df.shape[1]} columns such that', end=' ')
            print(f'{self.df.isnull().sum().sum()} NaNs are left - COMPLETE')

        elif self.imputation_handler == 'mean':
            print(f'Imputing NaNs with mean')

            # drop rows where the 'Revenue' is unknown (NaNs)
            print(f'Dropping rows that have no information about revenue', end=' - ')
            rows = self.df.shape[0]
            self.df.drop(self.df[self.df['Revenue'].isnull()].index, inplace=True)
            print(f'{rows - self.df.shape[0]} rows dropped - COMPLETE')

            # introduce new column that describes revenue ranges
            range_cond = [(self.df['Revenue'] <= 1e6),
                          (self.df['Revenue'] > 1e6) & (self.df['Revenue'] <= 1e7),
                          (self.df['Revenue'] > 1e7) & (self.df['Revenue'] <= 1e8),
                          (self.df['Revenue'] > 1e8) & (self.df['Revenue'] <= 1e9),
                          (self.df['Revenue'] > 1e9)]
            self.df['Revenue Range'] = np.select(range_cond, [0, 1, 2, 3, 4])

            # store sector column to add it after groupby
            sector_col = self.df['Sector']

            # group by sector and then revenue range and impute nans with means
            nans_count = self.df.isnull().sum().sum()
            self.df = self.df.groupby(['Sector', 'Revenue Range']).transform(lambda x: x.fillna(x.mean()))
            nans_count_after = self.df.isnull().sum().sum()
            print(f'{nans_count - nans_count_after} NaNs imputed', end=' - ')
            print(f'{nans_count_after} NaNs left - COMPLETE')

            # add sector column again
            self.df['Sector'] = sector_col

            # if there are still NaNs left, remove the corresponding row(s)
            if nans_count_after > 0:
                print(f'Dropping rows with remaining NaNs', end=' - ')
                row_count = self.df.shape[0]
                self.df = self.df.dropna(axis=0)
                print(f'{row_count - self.df.shape[0]} row(s) dropped', end=' - ')
                print(f'{self.df.isnull().sum().sum()} NaNs left - COMPLETE')

        self.__print_sep(60, '~', '\n')

    def handle_outliers(self):
        self.__print_header('HANDLE OUTLIERS')

        if 'quantile' in self.outlier_handler:
            rows_before = self.df.shape[0]
            cut_lower = self.config_dict[self.year]['cut_lower']
            cut_upper = self.config_dict[self.year]['cut_upper']
            # drop rows with an unnaturally high price variance
            print(f'Dropping rows with a price variance outside the {cut_lower} - {cut_upper} quantile range',
                  end=' - ')
            col = f'{self.year + 1} PRICE VAR [%]'
            outs = self.df[col].between(self.df[col].quantile(cut_lower),
                                        self.df[col].quantile(cut_upper))
            self.df.drop(self.df[~outs].index, inplace=True)
            print(f'{rows_before - self.df.shape[0]} rows dropped - COMPLETE')

        if 'IForest' in self.outlier_handler:
            print('Using Isolation Forest to detect outliers', end=' - ')
            contamination = self.config_dict[self.year]['IForest_contamination']
            clf = IForest(contamination=contamination,
                          random_state=42,
                          n_jobs=-1)
            clf.fit(self.df.values)
            y_pred = clf.predict(self.df.values)
            idx_y_pred = [i for i in range(self.df.shape[0]) if y_pred[i] == 0]
            self.df = self.df.iloc[idx_y_pred, :]
            print(f'{sum(y_pred)} outliers removed - COMPLETE')

        self.__print_sep(60, '~', '\n')

    def scale_data(self):
        self.__print_header('SCALING DATA')

        print(f'Scaling data', end=' - ')
        sector_col = self.df['Sector'] if 'Sector' in self.df.columns else None
        scaler = StandardScaler()
        np_scaled = scaler.fit_transform(self.df)
        self.df = pd.DataFrame(np_scaled,
                               columns=self.df.columns,
                               index=self.df.index)

        if sector_col is not None:
            self.df['Sector'] = sector_col
        print('COMPLETE')

    def drop_var_col(self):
        self.df.drop(columns=[f'{self.year + 1} PRICE VAR [%]'], inplace=True)

    def convert_join(self):
        # convert to float32
        self.df = self.df.astype('float32')

        # join 'Class' column
        self.df = self.df.join(self.class_col)

        self.__print_summary(f'Final DataFrame shape: {self.df.shape}')

    def map_sector_inv(self):
        mapper = {i: sector for i, sector in enumerate(self.sector_mapper)}
        self.df['Sector'] = self.df['Sector'].apply(lambda x: mapper[x])

    def get_intersecting_columns(self):
        df1 = self.dfs[0].columns
        df2 = self.dfs[1].columns
        df3 = self.dfs[2].columns
        df4 = self.dfs[3].columns
        df5 = self.dfs[4].columns

        self.intersec_cols = df1 & df2 & df3 & df4 & df5
        print(f'Found {len(self.intersec_cols)} intersecting columns!')

    def concat_intersecting_dfs(self):
        df1 = self.dfs[0][self.intersec_cols]
        df2 = self.dfs[1][self.intersec_cols]
        df3 = self.dfs[2][self.intersec_cols]
        df4 = self.dfs[3][self.intersec_cols]
        df5 = self.dfs[4][self.intersec_cols]

        df = pd.concat([df1, df2, df3, df4, df5])
        print(f'Concatenated DataFrame into shape: {df.shape}')

        return df

    def __print_summary(self, info: str):
        self.__print_sep()
        print(info)
        self.__print_sep(60, '~', '\n')

    def __print_sep(self, n: int = 60, c: str = '-', nl: str = ''):
        print(n * c + nl)

    def __print_header(self, header: str):
        rem = 60 - 25 - 2 - len(header)
        print(f'{25 * "~"} {header} {rem * "~"}')

# Compare approaches

## Set config dictionary

In [8]:
config_dict = {
    2014: {
        'nans_thres': 0.08, 
        'zeros_thres': 0.08,
        'cut_lower': 0.01,
        'cut_upper': 0.99,
        'IForest_contamination': 0.02
    },
    2015: {
        'nans_thres': 0.10, 
        'zeros_thres': 0.08,
        'cut_lower': 0.01,
        'cut_upper': 0.99,
        'IForest_contamination': 0.02
    },
    2016: {
        'nans_thres': 0.16, 
        'zeros_thres': 0.08,
        'cut_lower': 0.01,
        'cut_upper': 0.99,
        'IForest_contamination': 0.02
    },
    2017: {
        'nans_thres': 0.16, 
        'zeros_thres': 0.08,
        'cut_lower': 0.01,
        'cut_upper': 0.99,
        'IForest_contamination': 0.02
    },
    2018: {
        'nans_thres': 0.08, 
        'zeros_thres': 0.08,
        'cut_lower': 0.01,
        'cut_upper': 0.99,
        'IForest_contamination': 0.02
    }
}

## Approach 1

1.   *NaN threshold*
2.   *zero threshold*
3.   *mean imputation*
4.   *outlier quantile cut*



In [15]:
dataset_1 = DataSet(config_dict=config_dict,
                    nan_handler = ['threshold'],
                    zero_handler = ['threshold'],
                    imputation_handler = 'mean',
                    outlier_handler = ['quantile'])

# generate DataFrame
df_1 = dataset_1.create_dataset()

# use full df for cross validation vanilla test
X_1, y_1 = df_1.drop(columns=['Class']), df_1['Class']

############################################################

~~~~~~~~~~~~~~~~~~~~~~~~~ LOADING 2014 ~~~~~~~~~~~~~~~~~~~~~
Loading 2014_Financial_Data.csv into a DataFrame - COMPLETE
Dropping rows with NaNs only - 0 rows dropped - COMPLETE
------------------------------------------------------------
Initial DataFrame shape: (3808, 224)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

~~~~~~~~~~~~~~~~~~~~~~~~~ HANDLING NANS ~~~~~~~~~~~~~~~~~~~~
Dropping columns with more than 8% NaNs - 89 columns dropped - COMPLETE
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

~~~~~~~~~~~~~~~~~~~~~~~~~ HANDLING ZEROS ~~~~~~~~~~~~~~~~~~~
Dropping columns with more than 8% zeros - 60 columns dropped - COMPLETE
------------------------------------------------------------
Total amount of columns dropped: 150 - Current shape: (3808, 74)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

~~~~~~~~~~~~~~~~~~~~~~~~~ IMPUTE NANS ~~~~~~~~~~~~~~~~~~~~~~
Imputing NaNs with mea

/var/folders/7k/9x94hbs904jgh5ymqv00n1rh0000gn/T/ipykernel_33321/2900163752.py:287: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  self.intersec_cols = df1 & df2 & df3 & df4 & df5


In [16]:
test_vanilla(X_1, y_1)

Accuracy for ETC: 0.7009426914891994
Accuracy for RFC: 0.7019196115297779
Accuracy for GBC: 0.6988923869599708
Accuracy for CBC: 0.7033840858211802
Accuracy for KNN: 0.6094914867204662
Accuracy for SGDC: 0.5520240968048267


## Approach 2


1.   *zero threshold*
2.   *replace zeros with NaNs*
3.   *KNN imputation*
4.   *outlier quantile cut*
5.   *Isolation Forest outlier handling*



In [11]:
dataset_2 = DataSet(config_dict=config_dict,
                    nan_handler = [],
                    zero_handler = ['threshold', 'replace'],
                    imputation_handler = 'KNN',
                    outlier_handler = ['quantile', 'IForest'])

# generate DataFrame
df_2 = dataset_2.create_dataset()

# use full df for cross validation vanilla test
X_2, y_2 = df_2.drop(columns=['Class']), df_2['Class']

############################################################

~~~~~~~~~~~~~~~~~~~~~~~~~ LOADING 2014 ~~~~~~~~~~~~~~~~~~~~~
Loading 2014_Financial_Data.csv into a DataFrame - COMPLETE
Dropping rows with NaNs only - 0 rows dropped - COMPLETE
------------------------------------------------------------
Initial DataFrame shape: (3808, 224)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



IndexError: list index out of range

In [12]:
test_vanilla(X_2, y_2)

NameError: name 'X_2' is not defined

## Approach 3

1.   *replace zeros with NaNs*
3.   *XGB imputation*
4.   *outlier quantile cut*
5.   *Isolation Forest outlier handling*


In [13]:
dataset_3 = DataSet(config_dict=config_dict,
                    nan_handler = [],
                    zero_handler = ['replace'],
                    imputation_handler = 'XGB',
                    outlier_handler = ['quantile', 'IForest'])

# generate DataFrame
df_3 = dataset_3.create_dataset()

# use full df for cross validation vanilla test
X_3, y_3= df_3.drop(columns=['Class']), df_3['Class']

############################################################

~~~~~~~~~~~~~~~~~~~~~~~~~ LOADING 2014 ~~~~~~~~~~~~~~~~~~~~~
Loading 2014_Financial_Data.csv into a DataFrame - COMPLETE
Dropping rows with NaNs only - 0 rows dropped - COMPLETE
------------------------------------------------------------
Initial DataFrame shape: (3808, 224)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



IndexError: list index out of range

In [14]:
test_vanilla(X_3, y_3)

NameError: name 'X_3' is not defined

## Approach 4

1.   *NaN threshold*
2.   *zero threshold*
3.   *replace zeros with NaNs*
4.   *NaN threshold again*
5.   *KNN imputation*
6.   *outlier quantile cut*
7.   *Isolation Forest outlier handling*

In [ ]:
dataset_4 = DataSet(config_dict=config_dict,
                    nan_handler = [],
                    zero_handler = ['replace'],
                    imputation_handler = 'XGB',
                    outlier_handler = ['quantile', 'IForest'])

# generate DataFrame
df_4 = dataset_4.create_dataset()

# use full df for cross validation vanilla test
X_4, y_4= df_4.drop(columns=['Class']), df_4['Class']

In [ ]:
test_vanilla(X_4, y_4)